In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Convolution2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.metrics import categorical_accuracy
from keras import regularizers
from tqdm import tqdm
from PIL import Image

import numpy as np
import pandas as pd
import random
import tensorflow as tf
import cv2

import matplotlib
import matplotlib.pyplot as plt

import os
print(os.listdir("../input"))


In [ ]:
def crop_image_from_gray(img,tol=7):
    """
    Crop out black borders
    https://www.kaggle.com/ratthachat/aptos-updated-preprocessing-ben-s-cropping
    """  
    
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mask = gray_img>tol        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):
            return img
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img


def circle_crop(img):   
    """
    Create circular crop around image centre    
    """    
    
    img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    
    return img 

def circle_crop_v2(img):
    """
    Create circular crop around image centre
    """
    img = cv2.imread(img)
    img = crop_image_from_gray(img)

    height, width, depth = img.shape
    largest_side = np.max((height, width))
    img = cv2.resize(img, (largest_side, largest_side))

    height, width, depth = img.shape

    x = int(width / 2)
    y = int(height / 2)
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)

    return img

In [ ]:
# load all the image names in train images file 
img_size = 300
batch_size=32

from keras_preprocessing.image import ImageDataGenerator
def append_ext(fn):
    return fn+".png"

def convert_int(i):
    return str(i)

#image preprocessing
def preprocess_image(image_path):
    
    image = cv2.imread(image_path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    #ret3,image = cv2.threshold(image, 129, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    image = cv2.resize(image, (img_size, img_size))
    image = cv2.addWeighted(image,4, cv2.GaussianBlur(image,(0,0),img_size/10),-4 ,128)
    plt.imshow(image)
#    cv2.applyColorMap(im, cv2.COLORMAP_JET)
    return image

testdf=pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
testdf["id_code"]=testdf["id_code"].apply(append_ext)
testdf["diagnosis"]=testdf["diagnosis"].apply(convert_int)
image_path = "color_images_test"
#image_path = "../input/images-test/gray_images_test/gray_images_test"
N = testdf.shape[0]
x_test = np.empty((N, 300, 300, 3), dtype=np.uint8)
for i, image_id in enumerate(tqdm(testdf['id_code'])):
    x_test[i, :, :, :] = preprocess_image(f'../input/aptos2019-blindness-detection/test_images/{image_id}')
    

In [ ]:
# from keras.applications import ResNet50
# resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
# resNet50 = ResNet50(include_top = False, pooling = 'avg', weights = resnet_weights_path,classes=5)
# model = Sequential()
# model.add(resNet50)
# model.add(Dense(5, activation='softmax',activity_regularizer=regularizers.l1(0.001)))
# model.summary()
# model.load_weights('../input/graymodel/best_aptos_model (2).hdf5')

# from keras.applications import VGG16
# VGG16_weights_path = '../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
# vgg16 = VGG16(include_top=False,input_shape=(300, 300, 3),weights = VGG16_weights_path,classes=5)
# model = Sequential()
# model.add(vgg16)
# model.add(Flatten())
# model.add(Dense(256, activation='relu',activity_regularizer=regularizers.l2(0.001)))
# model.add(Dense(256, activation='relu',activity_regularizer=regularizers.l2(0.001)))
# model.add(Dropout(0.25))
# model.add(Dense(5, activation='softmax',activity_regularizer=regularizers.l1(0.001)))
# model.summary()
# model.load_weights('../input/colormodel/color_model.hdf5')

from keras.applications import InceptionResNetV2
inceptionResNetV2_path = '../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
inceptionResNetV2 = InceptionResNetV2(include_top=False,input_shape=(300, 300, 3),weights = inceptionResNetV2_path,classes=5)

model = Sequential()
model.add(inceptionResNetV2)
model.add(Flatten())
model.add(Dense(256, activation='relu',activity_regularizer=regularizers.l2(0.001)))
model.add(Dense(256, activation='relu',activity_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax',activity_regularizer=regularizers.l1(0.001)))
model.summary()
# model.load_weights('../input/inceptionResNetV2/inceptionResNetV2.hdf5')


In [ ]:

test_datagen=ImageDataGenerator(
    rescale=1./255.,
#     horizontal_flip=True,
# #     fill_mode='nearest',
#     zca_whitening=True,
)
test_generator = test_datagen.flow(
    x_test,
    batch_size=batch_size
)

test_generator.reset()
test_pred=model.predict_generator(test_generator,steps=len(test_generator),verbose=1)



In [ ]:
# #TTA
# tta_steps = 10
# predictions = []

# for i in tqdm(range(tta_steps)):
#     preds = model.predict_generator(test_datagen.flow(test_generator, batch_size=batch_size, shuffle=False), steps = len(validation_data)/batch_size)
#     predictions.append(preds)

In [ ]:
#test_pred = np.mean(test_pred, axis=0)
predicted_class=np.argmax(test_pred,axis=1)
print(predicted_class[:10])
import collections
print(collections.Counter(predicted_class))

In [ ]:
sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = predicted_class.astype(int)
sample.to_csv("submission.csv", index=False)